In [23]:
# En-tete

import os
os.chdir("/cal/homes/arenault/workspace/PAF_ScrapIngress")
import csv
import math as m

In [1]:
    # Dichotomie 2D\n,

    # La fonction prend en entree les limites de la zone initiale, la longueur de la zone finale, les coordonnées [x,y] du portail.
    # Elle renvoie l'ID de la zone dans laquelle se trouve le portail; -1 si le portail n'est pas dans la zone scrappee.

    def dichotomie2D(xmin,xmax,ymin,ymax,precision,portail):
        xp,yp = portail[0],portail[1]
        ID = 0
        xc,yc = (xmax-xmin)/2+xmin,(ymax-ymin)/2+ymin 
        if (xp > xmax or xp < xmin or yp > ymax or yp < ymin):       # Le portail est hors limite, on renvoie donc -1
            return -1
        while (ymax-ymin > precision and xmax-xmin > precision):     # Le portail etant dans les limites, on effectue une dichotomie
            if (xp <= xc and yp >= yc):                              # Portail est dans la zone superieure gauche
                ID = 10*ID+1
                xmax,ymin = xc,yc
                xc,yc = xc-(xmax-xmin)/2,yc+(ymax-ymin)/2
            elif (xp > xc and yp > yc):                              # Portail est dans la zone superieure droite
                ID = 10*ID+2
                xmin,ymin = xc,yc
                xc,yc = xc+(xmax-xmin)/2,yc+(ymax-ymin)/2
            elif (xp < xc and yp < yc):                              # Portail est dans la zone inferieure gauche
                ID = 10*ID+3
                xmax,ymax = xc,yc
                xc,yc = xc-(xmax-xmin)/2,yc-(ymax-ymin)/2
            elif (xp >= xc and yp <= yc):                            # Portail est dans la zone inferieure droite
                ID = 10*ID+4
                xmin,ymax = xc,yc
                xc,yc = xc+(xmax-xmin)/2,yc-(ymax-ymin)/2
        return ID

In [2]:
xmin,xmax,ymin,ymax = 0,4,0,4
precision = 1
portails = [[i+0.5,j+0.5] for i in range(4) for j in range(4)]
portails.append([0,7])

[dichotomie2D(xmin,xmax,ymin,ymax,precision,p) for p in portails]

[33, 31, 13, 11, 34, 32, 14, 12, 43, 41, 23, 21, 44, 42, 24, 22, -1]

In [33]:
# Conversion de la latitude, longitude en coordonnées cartésiennes

# La fonction prend en entree latitude et longitude d'un point de référence (l'origine de notre repère) ainsi que la
# latitude et la longitude du point qui nous interesse.
# En sortie on obtient les coordonnees (x,y) de ce point par rapport à l'origine en entrée.
# on utilise la formule "Haversine"

def degreToRad(lat):
    return lat*m.pi/180

def conversion(origin_lat,origin_long,latitude,longitude):
    origin_lat,origin_long = degreToRad(origin_lat),degreToRad(origin_long) 
    latitude, longitude = degreToRad(latitude),degreToRad(longitude) 
    dlat = latitude-origin_lat
    dlong = longitude-origin_long
    a = m.sin(dlat/2)**2 + m.cos(origin_lat)*m.cos(latitude)*m.sin(dlong/2)**2
    c = 2*m.atan2(m.sqrt(a),m.sqrt(1-a))
    c2 = 2*m.asin(a**(0.5))
    radius = 6371   #rayon de la Terre en km
    print(c,c2)
    return c*radius


(0.9677478736471431, 0.9677478736471432)
6165.52170301
6454.0403915


In [14]:
# ajout de l'id geographique dans le fichier ficIn

ficIn = "./portails.csv"

def id_geographique(ficIn):
    with open(ficIn,'r+') as fic:
        csv_reader = csv.reader(fic, delimiter = ';')
        for line in csv_reader:
            portal = line[0].split(',')
            ID = dichotomie2D(xmin,xmax,ymin,ymax,precision,portal)
            csv_writer = csv.writer(fic, delimiter = ';')
            csv_writer.writerow(line + [ID])

In [9]:
# Test de l'id

ficIn = "./portails.csv"
ficOut = "./test_id.csv"

with open(ficIn, 'r') as ficIn:
    with open(ficOut, 'w') as ficOut:
        csv_reader = csv.reader(ficIn, delimiter = ';')
        csv_writer = csv.writer(ficOut, delimiter = ';')
        k = 0
        for line in csv_reader:
            if (k<1000):
                csv_writer.writerow(line)
            k += 1
